In [1]:
import requests
from urllib.parse import quote
import base64


In [2]:
class TippiLink:
    def __init__(self, username, password, host="192.168.0.1"):
        self._username = username
        self._password = password
        self._host = host

    def _get_headers(self):
        headers = {
            'Referer': 'http://%s/userRpm/MenuRpm.htm' % self._host,
            'Authorization': self._auth_header(),
        }
        return headers

    def _auth_header(self):
        auth_bytes = bytes(self._username+':'+self._password, 'utf-8')
        auth_b64_bytes = base64.b64encode(auth_bytes)
        auth_b64_str = str(auth_b64_bytes, 'utf-8')

        auth_str = quote('Basic {}'.format(auth_b64_str))
        return auth_str

In [4]:
tl = TippiLink("luis", "AmE3024OT1", "192.168.66.32")


In [5]:
response = requests.get('http://%s/userRpm/%s.htm' % (tl._host, 'StatusRpm.htm'), headers=tl._get_headers())

content = response.content

In [80]:
ini = 'wlanHostPara = new Array('
fin = ');'

In [84]:
lista = str(content)[str(content).index(ini)+len(ini):str(content).index(fin)].replace('\\n', '').split(',')
print(lista[5])

 19


In [6]:
from bs4 import BeautifulSoup

In [7]:
soup = BeautifulSoup(content, "html.parser")

In [8]:
soup

<script language="javascript" type="text/javascript">
var statusPara = new Array(
2,
0,
16,
20000,
25789,
"4.4.5 Build 111209 Rel.40972n ",
"WA5210G v1 0816301C",
0,0 );
</script>
<script language="javascript" type="text/javascript">
var lanPara = new Array(
"F8-1A-67-CF-EF-82", "192.168.66.32", "255.255.255.0", 
0,0 );
</script>
<script language="javascript" type="text/javascript">
var wlanPara = new Array(
1,
"Sistelca_",
4,
2,
"F8-1A-67-CF-EF-82",
"192.168.66.32",
0,
5,
"",
0,0 );
</script>
<script language="javascript" type="text/javascript">
var statistList = new Array(
51681004, 154973784, 339900, 464694, 
0,0 );
</script>
<script language="javascript" type="text/javascript">
var wanPara = new Array(
0, "F8-1A-67-CF-EF-83", "0.0.0.0", 1, "0.0.0.0", 0, 0, "0.0.0.0", 2, 1, 0, "0.0.0.0 , 0.0.0.0", "", 0, 
0,0 );
</script>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="no-cache" http-equiv="pragma"/>
<meta content="wed, 26 Feb 1997 08:21:57 GMT" h

In [6]:
content

b'<SCRIPT language="javascript" type="text/javascript">\nvar statusPara = new Array(\n2,\n0,\n16,\n20000,\n241491,\n"4.4.5 Build 111209 Rel.40972n ",\n"WA5210G v1 0816301C",\n0,0 );\n</SCRIPT>\n<SCRIPT language="javascript" type="text/javascript">\nvar lanPara = new Array(\n"F8-1A-67-CF-EF-82", "192.168.66.32", "255.255.255.0", \n0,0 );\n</SCRIPT>\n<SCRIPT language="javascript" type="text/javascript">\nvar wlanPara = new Array(\n1,\n"Sistelca_",\n4,\n2,\n"F8-1A-67-CF-EF-82",\n"192.168.66.32",\n0,\n5,\n"",\n0,0 );\n</SCRIPT>\n<SCRIPT language="javascript" type="text/javascript">\nvar statistList = new Array(\n2313654213, 238308914, 5117549, 8037747, \n0,0 );\n</SCRIPT>\n<SCRIPT language="javascript" type="text/javascript">\nvar wanPara = new Array(\n0, "F8-1A-67-CF-EF-83", "0.0.0.0", 1, "0.0.0.0", 0, 0, "0.0.0.0", 2, 1, 0, "0.0.0.0 , 0.0.0.0", "", 0, \n0,0 );\n</SCRIPT>\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">\n<META http-equiv="pragma" content="no-cache">\n<